In [1]:
!pip3 install mtcnn
from numpy import save,load
import keras
from keras.preprocessing import image
from keras.engine import Layer
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
import os
import random
import tensorflow as tf
from mtcnn.mtcnn import MTCNN
import cv2
from matplotlib import pyplot


     |████████████████████████████████| 2.3MB 2.7MB/s 


Using TensorFlow backend.


In [2]:
#SET UP DRIVE IN COLAB
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/CNN2/Face'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1uu4MCOdjfOx0BLZ5kH2wt4bVozidgPTq/CNN2/Face


In [0]:
#LOAD ALL FILES IN /FACES/DATA wich contain 100x100 images of faces with closed and open eyes 
closed_eye_x=[]
photo=[]
i=0


for filename in os.listdir('./data/ClosedFace/'):
    print(filename)
    photo=img_to_array(load_img('./data/ClosedFace/'+filename))
    closed_eye_x.append(photo)
    #print(photo.shape)
    #if(i==2):
    #  break
    i=i+1
    print(i)

closed_eye_x=np.array(closed_eye_x,dtype=float)
save('./data/data_closed_face_x.npy',closed_eye_x) #saves the array as a binary for easier re-loading

open_eye_x=[]
photo=[]
i=0
for filename in os.listdir('./data/OpenFace/'):
    print(filename)
    photo=img_to_array(load_img('./data/OpenFace/'+filename))
    
    open_eye_x.append(photo)
    #if(i==2):
    #  break
    i=i+1
    print(i)

photo=[]
open_eye_x=np.array(open_eye_x,dtype=float)
save('./data/data_open_face_x.npy',open_eye_x)


In [0]:
closed_eye_x=load('./data/data_closed_face_x.npy') #loads binary files if they exist
open_eye_x=load('./data/data_open_face_x.npy')

In [6]:
#CROP THE EYES FROM EACH IMAGE, POSITION OF EYES GIVEN BY Multi-Task Cascaded Convolutional Networks model or MTCNN()
X=[]
sq=16
detector=MTCNN()
temp=[]
for sample in range(len(closed_eye_x)):
    temp=detector.detect_faces(closed_eye_x[sample])
    if(len(temp)!=0):
      x,y=temp[0]['keypoints']['left_eye']
      croppedL=closed_eye_x[sample,y-sq:y+sq,x-sq:x+sq,:]
      x,y=temp[0]['keypoints']['right_eye']
      croppedR=closed_eye_x[sample,y-sq:y+sq,x-sq:x+sq,:]
      X.append(croppedL)
      X.append(croppedR)
X=np.array(X)
print(X.shape) 
save('./data/data_closed_face_cropped_x.npy',X)

X=[]

temp=[]
for sample in range(len(open_eye_x)):
    temp=detector.detect_faces(open_eye_x[sample])
    if(len(temp)!=0):
      x,y=temp[0]['keypoints']['left_eye']
      croppedL=open_eye_x[sample,y-sq:y+sq,x-sq:x+sq,:]
      x,y=temp[0]['keypoints']['right_eye']
      croppedR=open_eye_x[sample,y-sq:y+sq,x-sq:x+sq,:]
      X.append(croppedL)
      X.append(croppedR)
X=np.array(X)
print(X.shape)
save('./data/data_open_face_cropped_x.npy',X)


(2228, 32, 32, 3)
(2454, 32, 32, 3)


In [0]:
closed_eye_x_crop=load('./data/data_closed_face_cropped_x.npy') #loads binary files if they exist
open_eye_x_crop=load('./data/data_open_face_cropped_x.npy')